In [1]:
import tensorflow as tf
import keras 
from keras.applications import DenseNet201
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
conv_base = DenseNet201(weights='imagenet', include_top = False, pooling = 'max', input_shape=(300,300,3))

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

In [3]:
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")

In [ ]:
# Training set has 472 samples

Training with k-fold Validation

In [ ]:
X.shape, (y == 0).sum(), (y == 1).sum()

((472, 300, 300, 3), 236, 236)

In [4]:
nos = X.shape[0]

samples = conv_base.predict(X)
labels = y.copy()

In [5]:
samples.shape, labels.shape

((472, 1920), (472,))

In [6]:
# To save memory
del X
del y

In [7]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

accuracies = []
f = 1

for train_index, test_index in skf.split(samples, labels):
        X_train = samples[train_index]
        X_test = samples[test_index]
        y_train = labels[train_index]
        y_test = labels[test_index]
      
        
        print('Fold {}'.format(f))
        clf = make_pipeline(StandardScaler(),SVC(gamma='auto'))
        clf.fit(X_train, y_train)
        
        test_predclass = clf.predict(X_test)
        
        accuracies.append([confusion_matrix(y_test,test_predclass)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,test_predclass)[1][1]/(y_test == 1.0).sum()])
        f+=1


Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8


In [8]:
zero_acc = 0
one_acc = 0

for i in accuracies:
  zero_acc+= i[0]
  one_acc+=i[1]

zero_acc/=8
one_acc/=8

print(zero_acc, one_acc)

0.7935344827586207 0.7931034482758621


Training on whole Training Dataset

In [9]:
xtest = np.load('gdrive/My Drive/pcb/xtest.npy')
ytest = np.load('gdrive/My Drive/pcb/ytest.npy')


In [11]:
testnos = ytest.shape[0]

test_samples = conv_base.predict(xtest)
test_labels = ytest.copy()

In [12]:
test_samples.shape, test_labels.shape

((552, 1920), (552,))

In [13]:
clftest = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clftest.fit(samples, labels)

        
test_predclass = clftest.predict(test_samples)

In [14]:
confusion_matrix(test_labels,test_predclass)[0][0]/(ytest == 0.0).sum(), confusion_matrix(test_labels,test_predclass)[1][1]/(ytest == 1.0).sum()

(0.847457627118644, 0.7829614604462475)

In [ ]:
# Saving model

# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

TypeError: ignored